In [68]:
import pandas as pd 
import os
os.chdir(r'C:\Users\nicho\OneDrive\Desktop\Bitcoin Regresssion Project')
BTC = pd.read_csv(r'BTC-USD.csv')
Margin = pd.read_csv(r'margin-statistics.csv')
Margin

,Year-Month,Debit Balances in Customers' Securities Margin Accounts,Free Credit Balances in Customers' Cash Accounts,Free Credit Balances in Customers' Securities Margin Accounts
0,2021-11,"918,598","235,638","211,623"
1,2021-10,"935,862","226,159","200,548"
2,2021-09,"903,117","229,279","205,564"
3,2021-08,"911,545","219,365","198,913"
4,2021-07,"844,324","215,463","195,762"
...,...,...,...,...
294,1997-05,"111,097","66,879",NaN
295,1997-04,"102,842","64,825",NaN
296,1997-03,"104,835","69,553",NaN
297,1997-02,"103,886","67,058",NaN


In [73]:
BTC1 = BTC.copy()
BTC1['Date'] = pd.to_datetime(BTC['Date'])
BTC1['Date'] = BTC1['Date'].dt.strftime('%Y-%m')
BTCdrop = BTC1.drop(['Open', 'High', 'Low', 'Symbol', 'Volume USD'], axis =1)
margin1 = Margin.rename(columns = {'Year-Month': 'Date'})
merged = pd.merge(BTCdrop, margin1, on = 'Date')
merged

,Date,Close,Volume BTC,Debit Balances in Customers' Securities Margin Accounts,Free Credit Balances in Customers' Cash Accounts,Free Credit Balances in Customers' Securities Margin Accounts
0,2020-04,7315.25,0.00,"524,696","180,942","217,187"
1,2020-04,7315.25,2237.13,"524,696","180,942","217,187"
2,2020-04,7369.60,2483.60,"524,696","180,942","217,187"
3,2020-04,7201.81,2333.34,"524,696","180,942","217,187"
4,2020-04,7336.96,3727.47,"524,696","180,942","217,187"
...,...,...,...,...,...,...
1642,2015-10,247.60,38.28,"513,941","174,437","163,210"
1643,2015-10,249.50,12.22,"513,941","174,437","163,210"
1644,2015-10,246.30,12.17,"513,941","174,437","163,210"
1645,2015-10,245.39,30.99,"513,941","174,437","163,210"
